# Подготовка текстов эбстрактов статей новостного архива SpectrumNews (https://www.spectrumnews.org/news-archive/) к анализу

Код ниже совершает следующие действия с базой текстов SpectrumNews.

 1. Импортирует базу в Python
 2. Очищает данные от лишних символов: знаков препинания и двойных пробелов
 3. Приводит все тексты к нижниму регистру
 4. Производит лемматизацию тектсов (приводит слова в изначальную форму)
 5. Удаляет стор-слова из текстов

Импортируем модуль pandas для работы с базами данных и импортируем необходимую базу данных.

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_excel('SpectrumNews_Publications.xlsx')

In [ ]:
df

База данных несколько отличается от базы из предыдущего задания. Для выполнения ДЗ_2 мною были выгружены данные по всем новостным статьям портала SpectrumNews, начиная с декабря 2007 года.

Прежде чем приступать к обработке всех эбстрактов, сперва на основе 1169 строки будут проведены все предполагаемые этапы и созданы функции для обработки.

In [ ]:
text=df.loc[2518, 'Abstract']
text

Очистка текстов от знаков препинания (кроме - в словах)и цифр.

In [ ]:
new_text=[]
for i in text:
    if i.isalpha() or i==' ' or i=='-':
        new_text.append(i)
''.join(new_text)

In [ ]:
def delete_all_but_text(text):
    new_text=[]
    for i in text:
        if i.isalpha() or i==' ' or i=='-':
            new_text.append(i)
    return ''.join(new_text)

In [ ]:
text=delete_all_but_text(text)
text

В тексте остались дефисы и появились двойные пробелы. Следующие функции преобразуют дефисы в пробелы и очищают текст от двойных пробелов.

In [ ]:
while '-' in text:
    text=text.replace('-', ' ')
text

In [ ]:
def turn_defs_to_space(text):
    while '  ' in text:
        text=text.replace('  ', ' ')
    return(text)

In [ ]:
while '  ' in text:
    text=text.replace('  ', ' ')
text

In [ ]:
'  ' in text

In [ ]:
def delete_doublespaces(text):
    while '  ' in text:
        text=text.replace('  ', ' ')
    return(text)

In [ ]:
delete_doublespaces(text)

В некоторых текстах не убирались символы, которые обозначают закрывающиеся ковычки. Коды далее убирают эти символы, но при этом оставляют символы апострофа в фамилиях: в базе встречаются такие случаи.

In [ ]:
while 'ʼ ' in text:
    text=text.replace('ʼ ', ' ')
text

In [ ]:
def delete_comas_after_text(text):
    while 'ʼ ' in text:
        text=text.replace('ʼ ', ' ')
    return(text)

In [ ]:
delete_comas_after_text(text)

Следующая функция приводит все буквы в тексте к нижниму регистру.

In [ ]:
text=text.lower()
text

Импорт модуля nltk для лемматизации текстов и получения списка стоп-слов.

In [ ]:
#pip install nltk
import nltk
#nltk.download()

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl=WordNetLemmatizer()

Токенизация и лемматизация текста.

In [ ]:
text_words=nltk.word_tokenize(text)
nltk.pos_tag(text_words)
lemmed_tokens=[]
for word, tag in nltk.pos_tag(text_words):
    wntag = tag[0].lower()
    wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
    lemmed_tokens.append(wnl.lemmatize(word, wntag) if wntag else word)
lemmed_tokens

In [ ]:
def lemmatize(text):
    text_words=nltk.word_tokenize(text)
    nltk.pos_tag(text_words)
    lemmed_tokens=[]
    for word, tag in nltk.pos_tag(text_words):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        lemmed_tokens.append(wnl.lemmatize(word, wntag) if wntag else word)
    return lemmed_tokens    

Импорт из модуля nltk списка стоп-слов и расширение его с собственными.

Тексты текущей базы данных содержат много аббревиатур и названий генов.

Названия некоторых генов состоят из обычных аббревиатур (FMRP, NLGN4 и др.) и я не стал добавлять их в список стоп-слов, предполагая, что даже очищенные от цифр, они могут дать какой-то результат в последующем анализе.

Некоторые названия генов в основном состояли из цифр и знаков препинания, имея лишь одну букву (22q13, 16p11.2). После очистки от цифр и знаков препинания оставались только буквы q и p. Они были добавлены в список стоп слов, потому что отдельные буквы не несут содержательной нагрузки.

In [ ]:
from nltk.corpus import stopwords
sw_eng = list(set(stopwords.words('english')))
len(sw_eng)

In [ ]:
our_sw=['q', 'p']
sw_eng.extend(our_sw)
len(sw_eng)

In [ ]:
sw_eng

Очищение от стоп-слов и склеивание токенов в единый текст типа str.

In [ ]:
tokens_no_stop_words=[]
for word in lemmed_tokens:
    if word not in sw_eng:
        tokens_no_stop_words.append(word)
    prepared_text=' '.join(tokens_no_stop_words)
prepared_text

In [ ]:
def delete_stop_words(lemmed_tokens):
    tokens_no_stop_words=[]
    for word in lemmed_tokens:
        if word not in sw_eng:
            tokens_no_stop_words.append(word)
        prepared_text=' '.join(tokens_no_stop_words)
    return prepared_text

Все необходимые функции были составлены, теперь будет произведена подготовка всех текстов базы данных.

Сперва обрабатываемые данные будут проверены на наличие миссингов, и в случае обнаружения они будут удалены.

In [ ]:
df['Abstract'].isna().sum()

In [ ]:
df[df['Abstract'].isna()]

In [ ]:
df=df.dropna()

Далее будут последовательно применены функции по преобразованию текстов. Результаты будут сохраняться в отдельном столбце.

In [ ]:
df['Abstract_prpd'] = df['Abstract'].apply(delete_all_but_text)
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd']=df['Abstract_prpd'].apply(delete_comas_after_text)
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd'] = df['Abstract_prpd'].apply(turn_defs_to_space)
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd'] = df['Abstract_prpd'].apply(delete_doublespaces)
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd']=df['Abstract_prpd'].str.lower()
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd']=df['Abstract_prpd'].apply(lemmatize)
df['Abstract_prpd']

In [ ]:
df['Abstract_prpd']=df['Abstract_prpd'].apply(delete_stop_words)
df['Abstract_prpd']

In [ ]:
df

Результаты подготовки данных сохраняются в excel.

In [ ]:
df.to_excel('SpectrumNews_Publications_Prepared.xlsx')